In [1]:
import json

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

## Question 4: Sending the Trip Data

Now we need to send the data to the green-trips topic

Read the data, and keep only these columns:

    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'

Now send all the data using this code:

producer.send(topic_name, value=message)

For each row (message) in the dataset. In this case, message is a dictionary.

After sending all the messages, flush the data:

producer.flush()

Use from time import time to see the total time

from time import time

t0 = time()

... your code

t1 = time()

took = t1 - t0

How much time did it take to send the entire dataset and flush?

In [17]:
import csv
import json
from kafka import KafkaProducer
from time import time

In [18]:
csv_file = 'data/green_tripdata_2019-10.csv'
topic_name = 'green-trips'
servers = 'localhost:9092'

t0 = time()

# create kafka producer
producer = KafkaProducer(
    bootstrap_servers=servers,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# read csv
with open(csv_file, 'r', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)

    for row in reader:
        # select target columns only
        selected = {
            "lpep_pickup_datetime": row['lpep_pickup_datetime'],
            "lpep_dropoff_datetime": row['lpep_dropoff_datetime'],
            "PULocationID": row['PULocationID'],
            "DOLocationID": row['DOLocationID'],
            "passenger_count": row['passenger_count'],
            "trip_distance": row['trip_distance'],
            "tip_amount": row['tip_amount']
        }

        # send each row in csv
        producer.send(topic_name, value=selected)

# flush and close connection
producer.flush()
producer.close()

t1 = time()

took = t1 - t0
print(took)

12.725700855255127


In [5]:
{'VendorID': '2', 'lpep_pickup_datetime': '2019-10-01 00:26:02', 
 'lpep_dropoff_datetime': '2019-10-01 00:39:58', 'store_and_fwd_flag': 'N', 
 'RatecodeID': '1', 'PULocationID': '112', 'DOLocationID': '196', 
 'passenger_count': '1', 'trip_distance': '5.88', 'fare_amount': '18', 
 'extra': '0.5', 'mta_tax': '0.5', 'tip_amount': '0', 'tolls_amount': '0', 
 'ehail_fee': '', 'improvement_surcharge': '0.3', 'total_amount': '19.3', 
 'payment_type': '2', 'trip_type': '1', 'congestion_surcharge': '0'}

total 85048
drwxr-xr-x@  3 romadv-mac  staff        96 Jul 12 17:48 .
drwxr-xr-x@ 16 romadv-mac  staff       512 Jul 13 09:20 ..
-rw-r--r--@  1 romadv-mac  staff  43540725 Jul 12 17:45 green_tripdata_2019-10.csv


In [20]:
import pandas as pd

df = pd.read_csv(csv_file)
df.shape

/var/folders/tg/pz9klzrx2s1dps15b_lgvhqr0000gn/T/ipykernel_82759/3674686674.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


(476386, 20)

In [21]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-10-01 00:26:02,2019-10-01 00:39:58,N,1.0,112,196,1.0,5.88,18.0,0.50,0.5,0.00,0.0,NaN,0.3,19.30,2.0,1.0,0.0
1,1.0,2019-10-01 00:18:11,2019-10-01 00:22:38,N,1.0,43,263,1.0,0.80,5.0,3.25,0.5,0.00,0.0,NaN,0.3,9.05,2.0,1.0,0.0
2,1.0,2019-10-01 00:09:31,2019-10-01 00:24:47,N,1.0,255,228,2.0,7.50,21.5,0.50,0.5,0.00,0.0,NaN,0.3,22.80,2.0,1.0,0.0
3,1.0,2019-10-01 00:37:40,2019-10-01 00:41:49,N,1.0,181,181,1.0,0.90,5.5,0.50,0.5,0.00,0.0,NaN,0.3,6.80,2.0,1.0,0.0
4,2.0,2019-10-01 00:08:13,2019-10-01 00:17:56,N,1.0,97,188,1.0,2.52,10.0,0.50,0.5,2.26,0.0,NaN,0.3,13.56,1.0,1.0,0.0
